# Blog Writing Autogen WorkflowThis notebook demonstrates how to build a small multi-agent workflow with [Autogen](https://github.com/microsoft/autogen) for creating a blog article on a user-provided topic. A dedicated writer agent produces the initial draft, while a critical-thinking reviewer agent evaluates the draft and requests improvements until the piece is satisfactory.

## 1. Environment SetupInstall Autogen and other optional helper packages if they are not already available in your environment. Feel free to skip this step when the dependencies are pre-installed.

In [ ]:
%pip install --quiet autogen openai tiktoken


## 2. Configure Large Language Model AccessUpdate the `config_list` below with the model(s) you want to use and ensure the relevant API keys are available as environment variables. The sample configuration uses the standard OpenAI-style environment variables (`OPENAI_API_KEY`) but you can adapt it to Azure OpenAI or other Autogen-supported providers.

In [ ]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

# Configure the models that the agents will use.
config_list = [
    {
        "model": "gpt-4o-mini",
        "api_key": None,  # Falls back to the OPENAI_API_KEY environment variable.
    }
]

llm_config = {
    "config_list": config_list,
    "temperature": 0.3,
}


## 3. Build the Writer and Critic AgentsThe writer agent focuses on producing a comprehensive blog post, while the critic agent evaluates clarity, factual grounding, structure, and overall persuasiveness. The critic will keep the conversation going until it is satisfied with the revisions.

In [ ]:
def build_agents():
    writer = AssistantAgent(
        name="blog_writer",
        system_message=(
            "You are an expert blog author. Compose engaging, well-structured blog posts that "
            "balance storytelling with actionable insights. Always include an introduction, "
            "several logically ordered sections, and a conclusion with clear takeaways."
        ),
        llm_config=llm_config,
    )

    critic = AssistantAgent(
        name="critical_reviewer",
        system_message=(
            "You are a rigorous critical-thinking editor. Review blog drafts carefully, checking for "
            "logical consistency, factual grounding, clarity, tone, and overall usefulness. Provide "
            "actionable feedback and request revisions until you are satisfied. When the draft meets "
            "high standards, reply with 'APPROVED' followed by a concise summary of strengths."
        ),
        llm_config=llm_config,
    )

    orchestrator = UserProxyAgent(
        name="workflow_coordinator",
        human_input_mode="NEVER",
        max_consecutive_auto_reply=10,
        system_message=(
            "Coordinate the conversation between the blog_writer and critical_reviewer. "
            "Begin by instructing the blog_writer to produce a draft about the given topic. "
            "Allow the critic to review and request revisions until it responds with 'APPROVED'. "
            "Once approved, ask the blog_writer for a polished final version and end the chat."
        ),
    )

    return orchestrator, writer, critic


## 4. Run the Workflow for a TopicThe helper function below spins up a group chat so the writer and critic can collaborate. The coordinator agent seeds the conversation with high-level instructions that include the input topic.

In [ ]:
def generate_blog(topic: str, min_word_count: int = 600):
    orchestrator, writer, critic = build_agents()

    # Share contextual instructions with the agents.
    initial_brief = f"""
You are collaborating on a blog post about the topic: "{topic}".

Requirements:
- The blog_writer must craft a narrative that is at least {min_word_count} words.
- The article should include data points, illustrative examples, and practical recommendations.
- The critical_reviewer should examine reasoning quality, call out unsupported claims, and push for improvements.
- Continue iterating until the critic replies with APPROVED.
- After approval, the blog_writer should deliver the final polished blog post preceded by 'FINAL BLOG'.
""".strip()

    groupchat = GroupChat(
        agents=[orchestrator, writer, critic],
        messages=[],
        max_round=12,
    )

    manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    response = orchestrator.initiate_chat(
        manager,
        message=initial_brief,
        summary_method="last_msg",
    )

    return response


## 5. Try It OutProvide a topic (and optionally tweak the word-count target) to run the workflow. The final cell prints the conversation transcript, including the draft iterations, critiques, and the final approved blog post.

In [ ]:
topic = "The Role of Edge AI in Sustainable Smart Cities"
conversation = generate_blog(topic, min_word_count=700)
conversation
